# Trabajo Práctico 1 : Reservas de Hotel

### Grupo 01 - Integrantes:
        Cabrera Rodríguez, Mateo 
        Gonzalez Alejo, Camila
        Bocanegra, Eduardo Martín

In [130]:
# Manipulacion de datos y funciones matemáticas
import pandas as pd
import numpy as np

# Visualizaciones
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

#modelos y métricas
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score#, precision_recall_curve, roc_curve,
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [131]:
ds = pd.read_csv('../datasets/hotels_train.csv')

In [132]:
# encuentro variables categoricas
categorical_columns = [col for col in ds.columns if ds[col].dtype == 'object' or ds[col].dtype == 'string']
print(categorical_columns)

['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 'reservation_status_date', 'id']


In [133]:
ds['children'].fillna(0, inplace=True)
ds['children'] = ds['children'].astype('int64')
ds = ds.drop(ds[ds['children'] > 8].index)
ds = ds.drop(ds[ds['babies'] > 8].index)
ds = ds.drop(ds[ds['adults'] == 0].index)
ds = ds.dropna(subset=['country'])

In [134]:
agents = ds['agent'].unique()
agents = np.delete(agents, 1)
ds['agent'].fillna(0, inplace=True)

companies = ds['company'].unique()
companies = np.delete(companies, 0)
ds['company'].fillna(0, inplace=True)

In [135]:
ds = ds.replace({'meal': {'BB': 0, 'HB': 1, 'FB': 2, 'SC': 3, 'Undefined': 4},# ver que hacer con undefined
                 'arrival_date_month': {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10,'November': 11, 'December': 12},
                 'reserved_room_type': {a : n for n, a in enumerate(["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", 'L'])},
                 'assigned_room_type': {a : n for n, a in enumerate(["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K",'L'])},
                 'deposit_type': {'No Deposit': 0, 'Refundable': 1, 'Non Refund': 2},
                 'agent': {agent : 1 for agent in agents},
                 'company': {company : 1 for company in companies}})

In [136]:
print(ds['assigned_room_type'].unique())

[ 3  0  5  4  7  2  6  8  1 10 11]


In [139]:
label_encoder = LabelEncoder()
ds['market_segment'] = label_encoder.fit_transform(ds['market_segment'])
ds['distribution_channel'] = label_encoder.fit_transform(ds['distribution_channel'])
ds['customer_type'] = label_encoder.fit_transform(ds['customer_type'])

In [140]:
categorical_columns = [col for col in ds.columns if ds[col].dtype == 'object' or ds[col].dtype == 'string']
print(categorical_columns)

['hotel', 'country', 'reservation_status_date', 'id']


In [142]:
X = ds.drop(['hotel', 'country', 'reservation_status_date', 'id', 'is_canceled'], axis=1) # eliminar country, reservation_status_date, company cuando se cargue el ds chp1
y = ds['is_canceled']

parametros = {'criterion': ['gini', 'entropy'],
              'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
              'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
              'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

kfold = KFold(n_splits=2, shuffle=True, random_state=42)
modelo = tree.DecisionTreeClassifier()
grid = GridSearchCV(modelo, parametros, cv=kfold, scoring='accuracy', n_jobs=-1)

grid.fit(X, y)

print(grid.best_score_)
print(grid.best_params_)


0.7838141077758627
{'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 6}
